In [1]:
import numpy as np

In [2]:
def normalize(x):
    norm1 = x / np.linalg.norm(x)
    return norm1

In [3]:
num_sitting = 18
# num_sleeping = 1

to_check = [2]

target_vec_1 = normalize(np.loadtxt("/home/sourav/Datasets/fmorph/male_sitting_" + str(to_check[0]) + "/skel.vec"))
# target_vec_2 = normalize(np.loadtxt("/home/sourav/Datasets/fmorph/male_sitting_" + str(to_check[1]) + "/skel.vec"))

dist_1 = []
# dist_2 = []

for i in range(1, num_sitting+1):
    if i == 8 or i in to_check:
        # data file not present. empty directory
        continue
    
    vec = np.loadtxt("/home/sourav/Datasets/fmorph/male_sitting_" + str(i) + "/skel.vec")
    vec = normalize(vec)
    
    score_1 = np.dot(vec, target_vec_1)
    # score_2 = np.dot(vec, target_vec_2)
    
    dist_1.append((score_1, i, "sitting"))
    # dist_2.append((score_2, i, "sitting"))

In [4]:
dist_1.sort(key=lambda x: x[0], reverse=True)
# dist_2.sort(key=lambda x: x[0], reverse=True)

Cosine similarity and finding distribution over regions is done in the following part.

In [5]:
root = "/home/sourav/Datasets/fmorph/"
folder_name = "male_sitting_"
file_name = "support.info"

from operator import add
from random import uniform

import random
from datetime import datetime
import time

In [91]:
def read_support_file(path):
    
    '''returns a vector of 0s and 1s'''
    vec = []
    with open(path, 'r') as fp:
        for line in fp:
            vec.append(int(line.strip().split()[1]))
    return vec

def read_region_dist():
    
    '''returns the common vec'''
    path = "/home/sourav/Datasets/fmorph/region_dist_vec.dat"
    vec = []
    with open(path) as fp:
        vec = fp.read().strip().split()
    vec = map(float, vec)
    vec = map(lambda x:x*0.2, vec)
    return vec

def sample_from(vec_1):
    '''samples regions from the vector'''
    random.seed(time.time())
    salience = [0] * 25
    for i in range(0, 10):
        tmp = []
        for val in vec_1:
            if val > 0.7:
                tmp.append(1)
            else:
                if uniform(0,1) <= val:
                    tmp.append(1)
                else:
                    tmp.append(0)
        salience = map(add, salience, tmp)
        
    salience = map(lambda x: x/10.0, salience)
    # print salience
    salience = map(lambda x: 1 if x >= 0.5 else 0, salience)
    
    return salience

In [106]:
k = 5
vec_1 = [0] * 25
for item in dist_1[:k]:
    path = root + folder_name + str(item[1]) + "/" + "support.info"
    tmp = map(lambda x: x *item[0], read_support_file(path))
    vec_1 = map(add, vec_1, tmp)

vec_1 = map(lambda x: 0.8 * (x/float(k)), vec_1)
main = read_region_dist()
vec_1 = map(add, vec_1, main)
vec_1 = sample_from(vec_1)

orig = read_support_file(root + folder_name + str(to_check[0]) + "/" + "support.info")

print vec_1
print orig

[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]


In [110]:
from matplotlib import pyplot as plt

joint_names_path = root + "joint_names.txt"

joint_names_list = []
with open(joint_names_path, 'r') as fp:
    joint_names_list = fp.read().strip().split()
    
plt.figure(1)
plt.subplot(211) # a 5X1 grid
plt.title('Approximated Salience Scores')
plt.stem(vec_1)
plt.xticks(range(0,25), [joint_names_list[i] for i in range(0,25)], rotation=90)

plt.subplot(212)
plt.title('Original Salience Scores')
plt.stem(orig)
plt.xticks(range(0,25), [joint_names_list[i] for i in range(0,25)], rotation=90)

plt.tight_layout()
plt.savefig(root + folder_name + str(to_check[0]) + "/" + "vector_diff.png")

In [9]:
k = 5
dist_1[:k]

[(0.81547712030911412, 5, 'sitting'),
 (0.81080333413396133, 10, 'sitting'),
 (0.76935420738048077, 3, 'sitting'),
 (0.74526475068829223, 7, 'sitting'),
 (0.72114783467933186, 16, 'sitting')]